<a href="https://colab.research.google.com/github/profitter261/Insurance-price-prediction/blob/main/Loan_prediction_ML_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('cleaned_dataset.csv')
df.head()

,age,bmi,children,charges_log,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest,age_bmi_interaction,smoker_bmi_interaction
0,1,2,0,9.73,0,1,0,0,1,38,2
1,0,3,1,7.45,1,0,0,1,0,54,0
2,1,3,3,8.40,1,0,0,1,0,84,0
3,1,1,0,10.00,1,0,1,0,0,33,0
4,1,2,0,8.26,1,0,1,0,0,64,0


In [ ]:
df = df.dropna(subset=['age_bmi_interaction', 'smoker_bmi_interaction'])

In [ ]:
X = df.drop(['charges_log', 'age_bmi_interaction', 'smoker_bmi_interaction'], axis=1)
y = df['charges_log']

In [ ]:
display(X.head())

,age,bmi,children,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,1,2,0,0,1,0,0,1
1,0,3,1,1,0,0,1,0
2,1,3,3,1,0,0,1,0
3,1,1,0,1,0,1,0,0
4,1,2,0,1,0,1,0,0


In [ ]:
display(y.head())

,charges_log
0,9.73
1,7.45
2,8.40
3,10.00
4,8.26


In [ ]:
# Train-Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error

xgbr = xgb.XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42, enable_categorical=True)
xgbr.fit(X_train, y_train)
print("XGBoost RMSE:", np.sqrt(mean_squared_error(y_test, xgbr.predict(X_test))))

XGBoost RMSE: 0.41281090211252003


In [ ]:
xgbr.predict(X_test)

array([ 8.898603 , 10.229921 ,  9.473868 ,  7.602146 ,  8.2706375,
        9.983748 ,  7.8539405, 10.751958 ,  8.962142 ,  8.314678 ,
        9.192584 ,  9.449694 ,  7.863069 ,  9.082167 ,  8.380934 ,
        8.7533455,  9.064811 ,  9.5053215,  8.141825 ,  8.986656 ,
        9.289101 , 10.647956 ,  9.957782 , 10.800466 , 10.592406 ,
        8.862053 ,  7.7584014,  9.424209 ,  9.150961 ,  6.9930453,
        9.484371 ,  8.635433 ,  7.911667 ,  7.5054083,  9.484371 ,
        8.849491 ,  7.68691  ,  8.166541 ,  8.853622 , 10.005264 ,
        8.541531 ,  9.171925 ,  9.157734 ,  8.371856 ,  8.0566225,
       10.727277 ,  8.141825 , 10.547995 ,  8.185971 ,  8.625956 ,
       10.650669 ,  9.022115 ,  7.905364 ,  8.384271 ,  8.632088 ,
        9.409351 ,  7.602146 ,  8.314678 ,  8.683591 ,  8.892913 ,
        8.080985 ,  7.863069 ,  9.484046 ,  7.051055 ,  9.260887 ,
        9.358089 ,  8.599791 ,  8.069923 ,  8.932082 ,  7.8539405,
        9.166217 ,  9.136463 ,  8.853622 ,  9.489604 ,  9.4693

In [ ]:
import joblib
joblib.dump(xgbr, 'xgboost_model.pkl')
from google.colab import files
files.download('xgboost_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np

preds = []
for i in range(100):  # bootstrap 100 times
    sample = X.sample(frac=1, replace=True)
    sample_y = y.loc[sample.index] # Get corresponding target values
    xgbr.fit(sample, sample_y)
    preds.append(xgbr.predict(X_test))

preds = np.array(preds)
mean_preds = preds.mean(axis=0)
ci_low = np.percentile(preds, 2.5, axis=0)
ci_high = np.percentile(preds, 97.5, axis=0)

In [ ]:
import statsmodels.api as sm

X = sm.add_constant(X_train)
model = sm.OLS(y_train, X).fit()

predictions = model.get_prediction(X_test)
ci = predictions.conf_int(alpha=0.05)  # 95% CI

In [ ]:
import plotly.graph_objs as go

# Get the list of features to plot against (excluding the constant)
features_to_plot = X_test.columns.drop('const')

for feature in features_to_plot:
    # Sort X_test and corresponding predictions/confidence intervals by the current feature
    X_test_sorted = X_test.sort_values(by=feature)
    sorted_indices = X_test_sorted.index
    mean_preds_sorted = mean_preds[X_test.index.get_indexer(sorted_indices)]
    ci_low_sorted = ci_low[X_test.index.get_indexer(sorted_indices)]
    ci_high_sorted = ci_high[X_test.index.get_indexer(sorted_indices)]

    fig = go.Figure()

    # Mean predictions
    fig.add_trace(go.Scatter(
        x=X_test_sorted[feature], y=mean_preds_sorted,
        mode='lines', name='Prediction'
    ))

    # Confidence Interval
    fig.add_trace(go.Scatter(
        x=X_test_sorted[feature], y=ci_high_sorted,
        mode='lines', line=dict(width=0),
        showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x=X_test_sorted[feature], y=ci_low_sorted,
        mode='lines', line=dict(width=0),
        fill='tonexty', fillcolor='rgba(0,100,80,0.2)',
        name='95% CI'
    ))

    fig.update_layout(
        title=f'Predictions with 95% Confidence Interval vs. {feature.capitalize()}',
        xaxis_title=feature.capitalize(),
        yaxis_title='charges_log'
    )

    fig.show()